<a href="https://colab.research.google.com/github/furlancad/Estudo-PMF/blob/main/Dashboard_de_FIIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://github.com/Trading-com-Dados/pmf/blob/main/logotipo-trading-com-dados.svg?raw=true" width = "300">

---

# **Obtenção e análise de dados de FIIs e criação de dashboard para tomada de decisão de investimento**

In [ ]:
import pandas as pd


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import matplotlib.pyplot as plt
import vizro.plotly.express as px
from vizro import Vizro
import vizro.models as vm

import yfinance as yf

# **Iniciar o crawling**

In [ ]:
driver = webdriver.Chrome()

Vamos utilizar este site: https://www.fundsexplorer.com.br/ranking

In [ ]:
driver.get('https://www.fundsexplorer.com.br/ranking')

# **Extração da tabela original**

In [ ]:
driver.find_element(By.XPATH,'//*[@id="upTo--default-fiis-table"]/div/table')

<selenium.webdriver.remote.webelement.WebElement (session="82a5e7d5b3ad553eae55a1ca51a7cb35", element="3BB5FF15640A9359DF1882BC15E0E565_element_277")>

In [ ]:
tb_din = driver.find_element(By.XPATH,'//*[@id="upTo--default-fiis-table"]/div/table').get_attribute('outerHTML')

In [ ]:
fiis = pd.read_html(tb_din, decimal = ',', thousands = '.')[0]

**Manipulação dos dados**

In [ ]:
fiis.shape

(438, 8)

In [ ]:
fiis.head()

,Fundos,Setor,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas
0,AAZQ11,Indefinido,1028256.82,0.95,"1,32 %","0,00 %",2.318222e+08,23739
1,ABCP11,Shoppings,71149.79,0.73,"0,93 %","0,52 %",4.326774e+08,17274
2,AFHI11,Papéis,916803.21,1.04,"0,00 %","0,00 %",3.210416e+08,27831
3,AGRX11,Outros,521008.43,NaN,"1,37 %","-0,99 %",NaN,0
4,AIEC11,Lajes Corporativas,834865.86,0.67,"2,33 %","1,91 %",4.012162e+08,16789


In [ ]:
fiis = fiis.replace({',':'.', '%':''}, regex = True)

In [ ]:
fiis[['Dividend Yield', 'Variação Preço']] = fiis[['Dividend Yield', 'Variação Preço']].apply(pd.to_numeric)

In [ ]:
fiis.dtypes

Fundos                   object
Setor                    object
Liquidez Diária (R$)    float64
P/VP                    float64
Dividend Yield          float64
Variação Preço          float64
Patrimônio Líquido      float64
Num. Cotistas             int64
dtype: object

In [ ]:
fiis = fiis.set_index('Fundos')

In [ ]:
fiis

,Setor,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas
Fundos,,,,,,,
AAZQ11,Indefinido,1028256.82,0.95,1.32,0.00,2.318222e+08,23739
ABCP11,Shoppings,71149.79,0.73,0.93,0.52,4.326774e+08,17274
AFHI11,Papéis,916803.21,1.04,0.00,0.00,3.210416e+08,27831
AGRX11,Outros,521008.43,NaN,1.37,-0.99,NaN,0
AIEC11,Lajes Corporativas,834865.86,0.67,2.33,1.91,4.012162e+08,16789
...,...,...,...,...,...,...,...
XTED11,Lajes Corporativas,17641.88,0.68,NaN,NaN,6.737820e+07,4098
YUFI11,Fundo de Desenvolvimento,255.00,0.78,0.00,0.00,5.038388e+07,60
YUFI11B,Indefinido,NaN,NaN,0.00,0.00,NaN,0


In [ ]:
fiis.fillna(0, inplace = True)

In [ ]:
fiis['FII'] = fiis.index
fiis['Setor'] = fiis['Setor'].astype(str)

In [ ]:
fiis.sort_values('P/VP', ascending = False)

,Setor,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas,FII
Fundos,,,,,,,,
PRTS11,Shoppings,0.00,1961.67,0.00,0.00,3.111607e+06,11,PRTS11
SRVD11,Indefinido,233.91,583.00,2.29,-9.79,-8.340963e+05,106,SRVD11
IBFF11,Fundo de Fundos,162549.50,4.49,0.00,0.00,9.855905e+06,3870,IBFF11
RBRM11,Fundo de Desenvolvimento,0.00,3.63,0.00,0.00,1.409052e+07,54,RBRM11
LOFT11B,Fundo de Desenvolvimento,0.00,2.98,0.00,0.00,1.966339e+07,1,LOFT11B
...,...,...,...,...,...,...,...,...
CPFO11,Fundo de Fundos,0.00,0.00,0.00,0.00,9.050911e+07,528,CPFO11
CPTI11,Fundo de Desenvolvimento,1350525.00,0.00,1.00,1.79,0.000000e+00,0,CPTI11
PPEI11,0,405058.07,0.00,4.40,4.99,0.000000e+00,0,PPEI11


# **Criação do Dashboard de FIIs**

In [ ]:
page = vm.Page(
    title = 'Dashboard de FIIs',
    components = [
        vm.Graph(id = 'scatter_dy', figure = px.scatter(fiis, x ='Num. Cotistas' , y ='Dividend Yield', color = 'Setor')),
        vm.Graph(id = 'bar_pvp', figure = px.bar(fiis, x ='FII' , y ='P/VP', color = 'Setor')),
        vm.Graph(id = 'bar_patliq', figure = px.bar(fiis, x ='FII' , y ='Patrimônio Líquido', color = 'Setor')),


    ],

    controls = [
        vm.Filter(column = 'Setor'),
        vm.Filter(column = 'Liquidez Diária (R$)', selector = vm.RangeSlider()),
        vm.Filter(column = 'Dividend Yield', selector = vm.RangeSlider()),
        vm.Filter(column = 'Num. Cotistas', selector = vm.RangeSlider())


    ]


)

dashboard = vm.Dashboard(pages = [page])

Vizro().build(dashboard).run()